/!\ Execution trop lente (plus de 10 minutes) /!\

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import random 
random.seed(42)

In [3]:
df = pd.read_csv('../data/Fraud.csv')

In [4]:
df = df.sample(n=200_000, random_state=42)

# Cleaning

In [5]:
from sklearn.model_selection import train_test_split
df = df.drop(['isFlaggedFraud','step', 'nameOrig', 'nameDest'], axis=1)
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

In [6]:
try : 
    from pycaret.clustering import *
except : 
    from pycaret.clustering import setup, evaluate_model, plot_model

import sklearn
sklearn.__version__

'0.23.2'

In [7]:
df_s = setup(data= df_train, ignore_features=['isFraud'], session_id=42)

,Description,Value
0,session_id,42
1,Original Data,"(140000, 7)"
2,Missing Values,False
3,Numeric Features,5
4,Categorical Features,1
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(140000, 10)"
9,CPU Jobs,-1


In [8]:
kmeans = create_model('kmeans', num_clusters=2)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.8154,58657.7534,1.0517,0,0,0


In [9]:
save_model(kmeans, 'kmeans_2')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True,
                                       features_todrop=['isFraud'],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='UNSUPERVISED_DUMMY_TARGET',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='most frequent',
                                 fill_value_categorical=None,
                                 fill_value_numer...
                 ('fix_perfect', 'passthrough'),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 

In [10]:
result = assign_model(kmeans)

In [11]:
result.head(10)


,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,Cluster
258133,PAYMENT,13854.15,37744.14,23889.99,0.00,0.00,0,Cluster 0
3356005,CASH_IN,261897.28,8708563.77,8970461.04,1219693.69,957796.41,0,Cluster 1
5219785,PAYMENT,6332.04,39974.00,33641.96,0.00,0.00,0,Cluster 0
671789,CASH_IN,131277.19,1007767.67,1139044.85,155803.86,24526.67,0,Cluster 0
381532,CASH_IN,19631.67,24772.00,44403.67,131171.83,111540.16,0,Cluster 0
2449190,CASH_OUT,206664.87,3524.00,0.00,0.00,206664.87,0,Cluster 0
3472983,PAYMENT,6675.67,0.00,0.00,0.00,0.00,0,Cluster 0
18273,PAYMENT,10070.05,20969.00,10898.95,0.00,0.00,0,Cluster 0
1651211,CASH_IN,188746.08,27213509.69,27402255.76,292897.74,104151.66,0,Cluster 1
664898,PAYMENT,2520.89,0.00,0.00,0.00,0.00,0,Cluster 0


In [12]:
result.loc[result['Cluster'] == 'Cluster 0', 'Cluster'] = 0

In [13]:
result.loc[result['Cluster'] == 'Cluster 1', 'Cluster'] = 1

In [14]:
result.head(10)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,Cluster
258133,PAYMENT,13854.15,37744.14,23889.99,0.00,0.00,0,0
3356005,CASH_IN,261897.28,8708563.77,8970461.04,1219693.69,957796.41,0,1
5219785,PAYMENT,6332.04,39974.00,33641.96,0.00,0.00,0,0
671789,CASH_IN,131277.19,1007767.67,1139044.85,155803.86,24526.67,0,0
381532,CASH_IN,19631.67,24772.00,44403.67,131171.83,111540.16,0,0
2449190,CASH_OUT,206664.87,3524.00,0.00,0.00,206664.87,0,0
3472983,PAYMENT,6675.67,0.00,0.00,0.00,0.00,0,0
18273,PAYMENT,10070.05,20969.00,10898.95,0.00,0.00,0,0
1651211,CASH_IN,188746.08,27213509.69,27402255.76,292897.74,104151.66,0,1
664898,PAYMENT,2520.89,0.00,0.00,0.00,0.00,0,0


In [15]:
df_train['Pred'] = result['Cluster'].values

In [16]:
df_train.head(10)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,Pred
258133,PAYMENT,13854.15,37744.14,23889.99,0.00,0.00,0,0
3356005,CASH_IN,261897.28,8708563.77,8970461.04,1219693.69,957796.41,0,1
5219785,PAYMENT,6332.04,39974.00,33641.96,0.00,0.00,0,0
671789,CASH_IN,131277.19,1007767.67,1139044.85,155803.86,24526.67,0,0
381532,CASH_IN,19631.67,24772.00,44403.67,131171.83,111540.16,0,0
2449190,CASH_OUT,206664.87,3524.00,0.00,0.00,206664.87,0,0
3472983,PAYMENT,6675.67,0.00,0.00,0.00,0.00,0,0
18273,PAYMENT,10070.05,20969.00,10898.95,0.00,0.00,0,0
1651211,CASH_IN,188746.08,27213509.69,27402255.76,292897.74,104151.66,0,1
664898,PAYMENT,2520.89,0.00,0.00,0.00,0.00,0,0


In [17]:
df_train.dtypes

type               object
amount            float64
oldbalanceOrg     float64
newbalanceOrig    float64
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
Pred               object
dtype: object

In [18]:
df_train['Pred'] = df_train['Pred'].astype('int64')

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(df_train['isFraud'], df_train['Pred'])

0.9328428571428572

In [20]:
pred = predict_model(kmeans, data=df_test)

In [21]:
pred.head(10)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,Cluster
3941027,PAYMENT,1004.54,5915.0,4910.46,0.00,0.00,0,Cluster 0
2408671,CASH_OUT,105458.55,0.0,0.00,148402.44,253860.99,0,Cluster 0
1283122,TRANSFER,384594.44,126519.0,0.00,513945.83,898540.27,0,Cluster 0
4042971,CASH_IN,406869.11,217746.0,624615.11,1697567.50,1290698.39,0,Cluster 0
5172924,PAYMENT,6260.37,41886.0,35625.63,0.00,0.00,0,Cluster 0
5809684,PAYMENT,3290.27,0.0,0.00,0.00,0.00,0,Cluster 0
1298221,CASH_OUT,165993.41,132426.0,0.00,0.00,165993.41,0,Cluster 0
3839531,CASH_OUT,150951.42,14583.0,0.00,0.00,150951.42,0,Cluster 0
1680267,TRANSFER,29103.91,0.0,0.00,110662.96,139766.87,0,Cluster 0
4054226,PAYMENT,21296.97,18986.0,0.00,0.00,0.00,0,Cluster 0


In [22]:
pred.loc[pred['Cluster'] == 'Cluster 0', 'Cluster'] = 0
pred.loc[pred['Cluster'] == 'Cluster 1', 'Cluster'] = 1

In [23]:
df_test['Pred'] = pred['Cluster'].values

In [24]:
df_test.head(10)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,Pred
3941027,PAYMENT,1004.54,5915.0,4910.46,0.00,0.00,0,0
2408671,CASH_OUT,105458.55,0.0,0.00,148402.44,253860.99,0,0
1283122,TRANSFER,384594.44,126519.0,0.00,513945.83,898540.27,0,0
4042971,CASH_IN,406869.11,217746.0,624615.11,1697567.50,1290698.39,0,0
5172924,PAYMENT,6260.37,41886.0,35625.63,0.00,0.00,0,0
5809684,PAYMENT,3290.27,0.0,0.00,0.00,0.00,0,0
1298221,CASH_OUT,165993.41,132426.0,0.00,0.00,165993.41,0,0
3839531,CASH_OUT,150951.42,14583.0,0.00,0.00,150951.42,0,0
1680267,TRANSFER,29103.91,0.0,0.00,110662.96,139766.87,0,0
4054226,PAYMENT,21296.97,18986.0,0.00,0.00,0.00,0,0


In [25]:
df_test['Pred'] = df_test['Pred'].astype('int64')

In [26]:
accuracy_score(df_test['isFraud'], df_test['Pred'])

0.9321166666666667